In [16]:
"""
SpeechT5 uses speaker-embeddings that can output several voice types.

| Text-to-speech Models          | Voice Styles     |      Size | Note |
| ------------------------------ | ---------------- | --------- | ---- |
| microsoft/speecht5_tts         | Female Voice     |    145 MB | English only, LJ_Speech dataset |

# TTS Datasets
- LJSpeech (13,100 English audio clips, 24 hours) https://huggingface.co/datasets/lj_speech
- MLS (Multilingual LibriSpeech, 8 languages, from LibriVox) https://huggingface.co/datasets/facebook/multilingual_librispeech
- VCTK (Voice Cloning Toolkit, 44 hours, 110 speakers) - https://huggingface.co/datasets/vctk
- Libri-TTS/LibriTTS-R (English only 585 hrs by Google) - https://huggingface.co/datasets/cdminix/libritts-r-aligned

Reference:
- SpeechT5 - https://huggingface.co/blog/speecht5
- Fine-tuning SpeechT5 for multilingual TTS - https://colab.research.google.com/drive/1i7I5pzBcU3WDFarDnzweIj4-sVVoIUFJ
"""

try:
    import transformers, datasets, accelerate, soundfile, speechbrain
except:
    print('Installing modules...')
    %pip install -q transformers datasets accelerate soundfile speechbrain

Installing modules...


/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xformers 0.0.22.post7 requires torch==2.1.0, but you have torch 2.1.2 which is incompatible.
audiocraft 1.2.0 requires torch==2.1.0, but you have torch 2.1.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [45]:
import torch
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech
from transformers import SpeechT5HifiGan
from datasets import load_dataset
class TTS_t5():
    """Use SpeechT5, which is ~585 MB,

    The sample rate used by SpeechT5 is always 16 kHz.
    """

    model_id = "microsoft/speecht5_tts"

    def __init__(self):
        self.processor = SpeechT5Processor.from_pretrained(self.model_id)
        self.model = SpeechT5ForTextToSpeech.from_pretrained(self.model_id)

    def tts(self, text):
        inputs = self.processor(text=text, return_tensors="pt")

        embeddings_ds = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
        embeddings_ds = embeddings_ds.to_pandas()
        # Dataset({features: ['filename', 'xvector'], num_rows: 7931})

        emb_selected = torch.tensor(list(embeddings_ds[embeddings_ds["filename"]=="cmu_us_bdl_arctic-wav-arctic_a0002"]["xvector"]))

        # speaker_embeddings = torch.tensor(embeddings_ds[7306]["xvector"]).unsqueeze(0)
        # speaker_embeddings = torch.tensor(emb_selected["xvector"]).unsqueeze(0)

        spectrogram = self.model.generate_speech(inputs["input_ids"], emb_selected)
        vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
        with torch.no_grad():
            speech = vocoder(spectrogram)


        # speech = self.model.generate_speech(inputs["input_ids"], emb_selected, vocoder=vocoder)
        return speech, 16000

    def match_voice(self):
        dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
        dataset = dataset.sort("id")
        example = dataset[40]

In [46]:
text = """
Today is a new day, full of new possibilities. \
Believe in yourself and your abilities, and remember that every accomplishment \
starts with a single step. Embrace challenges as opportunities for growth, \
and don't be afraid to try something new. You are capable of achieving greatness, \
and every day is a fresh chance to make it happen. So rise up, \
face the day with a positive attitude, and make today amazing!
"""
model_tts_t5 = TTS_t5()

waveform, sampling_rate = model_tts_t5.tts(text)

from IPython.display import Audio as IPythonAudio

IPythonAudio(waveform, rate=sampling_rate)

1


In [ ]:
from speecht5.utils import get_voices

ModuleNotFoundError: No module named 'speecht5'